## LangChain Agent, Tool, LLM

Based on:
- [Pinecone LangChain agents](https://www.pinecone.io/learn/series/langchain/langchain-agents/)

To use agents, we require three things:

- A base LLM
- A tool that we will be interacting with
- An agent to control the interaction.

### Auth

In [2]:
import google.auth
import os
import vertexai


from dotenv import load_dotenv, dotenv_values

In [3]:
langchain_api_key = os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
langchain_project_id = os.getenv('LANGCHAIN_PROJECT_ID')
langchain_api_key

'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'

In [4]:
project_id = os.getenv('PROJECT_ID')
region_id = os.getenv('REGION_ID')
llm_model_id = os.getenv('CHAT_MODEL')
embedding_model_id = os.getenv('EMB_MODEL')

In [5]:
config = {
    'project_id': project_id,
    'llm_model_id': llm_model_id,
    'embedding_model_id': embedding_model_id
}

# authenticate to GCP
creds, _ = google.auth.default(quota_project_id=config["project_id"])
print(creds)

### Models

- The use of VertexAI instead of ChatVertexAI uses the [ReAct COT notebook example from GCP](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/chain_of_thought_react.ipynb)

In [6]:
from langchain_google_vertexai import VertexAI, ChatVertexAI, VertexAIEmbeddings

vertexai.init(project=project_id, location=region_id)

# ChatVertexAI doesn't seem to work in the ReAct agent setting as well as VertexAI
# llm = ChatVertexAI(
#     credentials=creds,
#     model_name=config["llm_model_id"],
#     temperature=0 # to pick more exact answers for tools use-cases
# )

# VertexAI seems to work as well in the ReAct agent setting as ChatVertexAI
llm = VertexAI(
    credentials=creds,
    model_name=config["llm_model_id"],
    temperature=0
)

embeddings = VertexAIEmbeddings(
    credentials=creds, model_name=config["embedding_model_id"])


### Tracing

In [7]:
def trace(toggle):
    if toggle:
        os.environ['LANGCHAIN_TRACING_V2'] = 'true'
        os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
        os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
        os.environ['LANGCHAIN_PROJECT'] = langchain_project_id
    else:
        del os.environ['LANGCHAIN_TRACING_V2']
        del os.environ['LANGCHAIN_ENDPOINT']
        del os.environ['LANGCHAIN_API_KEY']
        del os.environ['LANGCHAIN_PROJECT']

In [8]:
trace(True)

In [9]:
from langsmith import Client

client = Client(api_key=langchain_api_key)

url = next(client.list_runs(project_name="default")).url
print(f"You can monitor your trace at: {url}")

You can monitor your trace at: https://smith.langchain.com/o/2b56632f-de5b-5167-8339-f61d01e2ecb2/projects/p/3491b5eb-5738-4dbd-b4f1-bd7ed41971d4/r/0e27fd8d-0d4b-4a8f-b21f-51758f993bca?trace_id=94f473ad-0f02-44e5-a313-9bd4256709ae&start_time=2024-05-23T22:29:20.973000


### Calculator tool

- Uses [LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html): a chain that interprets a prompt and executes python code to do math.

In [10]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm)
llm_math.prompt.format(question="What is 2*4?")

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${Question with math problem.}\n```text\n${single line mathematical expression that solves the problem}\n```\n...numexpr.evaluate(text)...\n```output\n${Output of running the code}\n```\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: What is 2*4?\n'

In [11]:
llm_math.invoke("What is 2*4?")

{'question': 'What is 2*4?', 'answer': 'Answer: 8'}

In [12]:
calculator_tool = Tool(
    name='Calculator',
    func=llm_math.run, # .run on top of an existing chain
    description='A tool that is useful when the user needs answers to math questions.'
)

#### Add to list of tools

In [13]:
# when giving tools to LLM, we must pass as list of tools
tools = [calculator_tool]

### Agent

So far we have an llm and a tool but not an agent. To use agents, we require three things:

- A base LLM
- A tool that we will be interacting with
- An agent to control the interaction

#### Zero-shot agents
- Zero-shot means the agent functions on the current action only — it has no memory.
- It uses the ReAct framework to decide which tool to use, based solely on the tool’s `description`.
- The ReAct standard prompt is [hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)
- ReAct can be thought about as an LLM that could cycle through Reasoning and Action steps. Enabling a multi-step process for identifying answers.

In [29]:
from langchain import hub
from langchain.agents import initialize_agent, create_structured_chat_agent, AgentType, AgentExecutor

# TODO: initialize_agent is deprecated
# this agent uses hwchase17/structured-chat-agent as the prompt
# agent = initialize_agent(
#     tools,
#     llm,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
# )

prompt = hub.pull('hwchase17/structured-chat-agent')

In [35]:
def get_structured_chat_agent(prompt, llm, tools):
    structured_chat_agent = create_structured_chat_agent(
        llm=llm,
        tools=tools,
        prompt=prompt
    )
    return AgentExecutor(
        agent=structured_chat_agent, 
        tools=tools, 
        verbose=True, 
        # memory=memory,
        # return_intermediate_steps=True,  # TODO: seems to confuse the agent
        handle_parsing_errors=True,
        max_iterations=5
    )


In [31]:
agent = get_structured_chat_agent(prompt, llm, tools)

#### Math questions should work

In [32]:
agent.invoke({'input': 'What is 17+32?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17+32"
}
```Answer: 49Action:
```
{
  "action": "Final Answer",
  "action_input": "The answer is 49."
}
```

> Finished chain.


{'input': 'What is 17+32?', 'output': 'The answer is 49.'}

In [21]:
agent.invoke({
    "input": "if Mary has four apples and Giorgio brings two and a half apple " \
        "boxes (apple box contains eight apples), how many apples do we " \
            "have?"})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Calculator",
  "action_input": "4 + 2.5 * 8"
}
```Answer: 24.0Action:
```json
{
  "action": "Final Answer",
  "action_input": "Mary and Giorgio have 24 apples in total."
}
```

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'Mary and Giorgio have 24 apples in total.'}

In [22]:
agent.invoke({
    "input":"If I had 2 shoes and 4 socks and two socks are enough for one shoe, how many socks do I not need?"})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Calculator",
  "action_input": "4 - 2"
}
```Answer: 2Action:
```json
{
  "action": "Final Answer",
  "action_input": "You do not need 2 socks."
}
```


> Finished chain.


{'input': 'If I had 2 shoes and 4 socks and two socks are enough for one shoe, how many socks do I not need?',
 'output': 'You do not need 2 socks.'}

##### Non-math questions should fail

In [25]:
agent.invoke({'input':'What is the weather tomorrow?'})



> Entering new AgentExecutor chain...
## Thought:

The user is asking about the weather for tomorrow. I can use the "Weather" tool to get the weather forecast for the user's location.

## Action:

```json
{
  "action": "Weather",
  "action_input": {
    "location": "user's location",
    "date": "tomorrow"
  }
}
```

## Observation:

(This will be filled in with the actual weather forecast once the tool is used.)
Weather is not a valid tool, try one of [Calculator].Action:
```json
{
  "action": "Final Answer",
  "action_input": "I'm sorry, I can't answer that question. I don't have access to the weather tool."
}
```

> Finished chain.


{'input': 'What is the weather tomorrow?',
 'output': "I'm sorry, I can't answer that question. I don't have access to the weather tool."}

### Fallback tool

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

fallback_llm_prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

fallback_llm_chain = LLMChain(llm=llm, prompt=fallback_llm_prompt)

fallback_llm_tool = Tool(
    name='Fallback Tool',
    func=fallback_llm_chain.run,
    description="use this tool for general purpose queries and logic that aren't Medicare questions or math questions"
)

### Add to list of tools

In [27]:
tools = [
    calculator_tool,
    fallback_llm_tool
]

### Reinitialize agent

In [36]:
agent = get_structured_chat_agent(prompt, llm, tools)

#### Non-math fallback questions should now work

In [38]:
agent.invoke({'input': 'What is the weather tomorrow?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is the weather tomorrow?"
}
```I do not have access to real-time information, including weather forecasts. To get the most accurate and up-to-date weather information for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am sorry, I do not have access to real-time information, including weather forecasts. To get the most accurate and up-to-date weather information for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station."
}
```

> Finished chain.


{'input': 'What is the weather tomorrow?',
 'output': 'I am sorry, I do not have access to real-time information, including weather forecasts. To get the most accurate and up-to-date weather information for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station.'}

#### Math questions should still work

In [39]:
agent.invoke({'input':'What is seventeen to the power of 3?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17^3"
}
```Answer: 4913Action:
```
{
  "action": "Final Answer",
  "action_input": "The answer is 4913."
}
```

> Finished chain.


{'input': 'What is seventeen to the power of 3?',
 'output': 'The answer is 4913.'}

#### Combination math and general questions should work

In [40]:
agent.invoke({'input':'A plant can grow 6 fruit per day. How many fruits grow in 30 days? What is this plant called?'})



> Entering new AgentExecutor chain...
Thought: The question asks for the number of fruits a plant can grow in 30 days and the name of the plant.
Action:
```
{
  "action": "Calculator",
  "action_input": "6 * 30"
}
```Answer: 180Action:
```
{
  "action": "Final Answer",
  "action_input": "A plant can grow 180 fruits in 30 days. I am sorry, but I cannot answer the second part of your question about the name of the plant as I do not have access to information about specific plant species."
}
```

> Finished chain.


{'input': 'A plant can grow 6 fruit per day. How many fruits grow in 30 days? What is this plant called?',
 'output': 'A plant can grow 180 fruits in 30 days. I am sorry, but I cannot answer the second part of your question about the name of the plant as I do not have access to information about specific plant species.'}

### Medicare tool

#### Data

In [41]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings

loader = PyPDFLoader("https://www.medicare.gov/Pubs/pdf/10050-medicare-and-you.pdf", extract_images=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(documents, embeddings)

#### Retriever

In [42]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

retriever = db.as_retriever(search_kwargs={'k':1}) # just one doc

#### Tool

In [43]:
from langchain.tools.retriever import create_retriever_tool

medicare_tool = create_retriever_tool(
    retriever,
    "Medicare Knowledge Base",
    "Query a retriever to get information about Medicare questions"
)

In [44]:
medicare_tool.invoke('What is Medicare part D?')

'9\nWhat are the parts of \nMedicare?\nPart A (Hospital Insurance) \nHelps cover:\n• Inpatient care in hospitals\n• Skilled nursing facility care\n• Hospice care\n• Home health care\nGo to pages 25–29. \nPart B (Medical Insurance)\nHelps cover:\n• Services from doctors and other health care providers\n• Outpatient care\n• Home health care\n• Durable medical equipment (like wheelchairs, walkers, \nhospital beds, and other equipment)\n• Many preventive services  (like screenings, shots or vaccines, \nand yearly “ Wellness” visits) \nGo to pages 29–55 . \nPart D (Drug coverage) \nHelps cover the cost of prescription drugs (including many recommended shots or vaccines).\nPlans that offer Medicare drug coverage (Part D) are run by \nprivate insurance companies that follow rules set by Medicare.\nGo to pages 79–90.'

### Add to list of tools

In [45]:
tools = [
    calculator_tool,
    fallback_llm_tool,
    medicare_tool
]

### Reinitialize agent

In [46]:
agent = get_structured_chat_agent(prompt, llm, tools)

#### Medicare questions should work

In [47]:
agent.invoke({'input': 'What is Medicare part A?'})



> Entering new AgentExecutor chain...
```json
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "What is Medicare Part A?"
  }
}
```9
What are the parts of 
Medicare?
Part A (Hospital Insurance) 
Helps cover:
• Inpatient care in hospitals
• Skilled nursing facility care
• Hospice care
• Home health care
Go to pages 25–29. 
Part B (Medical Insurance)
Helps cover:
• Services from doctors and other health care providers
• Outpatient care
• Home health care
• Durable medical equipment (like wheelchairs, walkers, 
hospital beds, and other equipment)
• Many preventive services  (like screenings, shots or vaccines, 
and yearly “ Wellness” visits) 
Go to pages 29–55 . 
Part D (Drug coverage) 
Helps cover the cost of prescription drugs (including many recommended shots or vaccines).
Plans that offer Medicare drug coverage (Part D) are run by 
private insurance companies that follow rules set by Medicare.
Go to pages 79–90.Action:
```json
{
  "action": "Final Answer",
 

{'input': 'What is Medicare part A?',
 'output': 'Medicare Part A is hospital insurance that helps cover inpatient care in hospitals, skilled nursing facility care, hospice care, and home health care.'}

#### Math questions should still work

In [48]:
agent.invoke({'input': 'What is 17+4?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17+4"
}
```Answer: 21Action:
```
{
  "action": "Final Answer",
  "action_input": "I am sorry, I cannot answer that question. I am not able to access the internet or any external resources."
}
```

> Finished chain.


{'input': 'What is 17+4?',
 'output': 'I am sorry, I cannot answer that question. I am not able to access the internet or any external resources.'}

#### Combination math and general questions should still work

In [49]:
agent.invoke({'input': 'What is the capital of Hungary? What is 99 minus 23??'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "The capital of Hungary is Budapest. 99 minus 23 equals 76."
}
```

> Finished chain.


{'input': 'What is the capital of Hungary? What is 99 minus 23??',
 'output': 'The capital of Hungary is Budapest. 99 minus 23 equals 76.'}

In [50]:
agent.invoke({'input': 'I have a son aged 10 years. I have a daughter that is 20 years older than the son. What Medicare benefits do my two dependents enjoy?'})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "What Medicare benefits do dependents enjoy?"
  }
}
``` 
SECTION 2:  Find out what Medicare covers27
Medicare also covers inpatient respite care, which is care you get in a Medicare-
approved facility so that your usual caregiver (family member or friend) can rest. You can stay up to 5 days each time you get respite care. 
After 6 months, you can continue to get hospice care as long as the hospice 
medical director or hospice doctor recertifies (at a face-to-face meeting) that you’re still terminally ill. 
You pay:
• Nothing for hospice care. 
• A copayment  of up to $5 per prescription for outpatient drugs for pain and 
symptom management. 
• Five percent of the Medicare-approved amount  for inpatient respite care. 
Original Medicare will be billed for your hospice care, even if you’re in a Medicare Advantage Plan . When you get hospice care, your Medicare 

{'input': 'I have a son aged 10 years. I have a daughter that is 20 years older than the son. What Medicare benefits do my two dependents enjoy?',
 'output': "I'm sorry, but Medicare does not cover dependents. Medicare is a federal health insurance program for people who are 65 or older, certain younger people with disabilities, and people with End-Stage Renal Disease (permanent kidney failure). If you are looking for health insurance for your children, you may want to consider a private health insurance plan."}

### Agent with Memory
- According to [LangChain AgentTypes](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/) we need an agent that can support both:
  - History
  - Multi tools
- That seems to be [Structured Chat](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/structured_chat/)

#### Memory

In [51]:
memory_key='chat_history'

In [52]:
from langchain.memory import ConversationBufferWindowMemory

agent_memory = ConversationBufferWindowMemory(
    memory_key=memory_key,
    k=5,
    return_messages=True
)

#### Agent

- Same as before but this time, with memory

In [53]:
def get_structured_chat_agent_with_memory(prompt, llm, tools, memory):
    structured_chat_agent = create_structured_chat_agent(
        llm=llm,
        tools=tools,
        prompt=prompt
    )
    return AgentExecutor(
        agent=structured_chat_agent, 
        tools=tools, 
        verbose=True, 
        memory=memory,
        # return_intermediate_steps=True,  # TODO: seems to confuse the agent
        handle_parsing_errors=True,
        max_iterations=5
    )

In [55]:
agent = get_structured_chat_agent_with_memory(prompt, llm, tools, agent_memory)

#### Set name in memory

In [56]:
agent.invoke({"input": "My name is Bob!"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hi Bob, it's nice to meet you!"
}
```


> Finished chain.


{'input': 'My name is Bob!',
 'chat_history': [],
 'output': "Hi Bob, it's nice to meet you!"}

#### See name in memory

In [57]:
agent_memory.buffer_as_messages

[HumanMessage(content='My name is Bob!'),
 AIMessage(content="Hi Bob, it's nice to meet you!")]

#### Use name in memory

In [58]:
agent.invoke({"input": "What is my name?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Your name is Bob."
}
```

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is Bob!'),
  AIMessage(content="Hi Bob, it's nice to meet you!")],
 'output': 'Your name is Bob.'}

#### Set age in memory

In [59]:
agent.invoke({'input': 'I am 59 years old'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "You are 59 years old."
}
```

> Finished chain.


{'input': 'I am 59 years old',
 'chat_history': [HumanMessage(content='My name is Bob!'),
  AIMessage(content="Hi Bob, it's nice to meet you!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Bob.')],
 'output': 'You are 59 years old.'}

#### Use age in Medicare question

In [61]:
agent.invoke({'input': 'Am I eligible for Medicare based on my age?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "Am I eligible for Medicare based on my age of 59?"
  }
}
```SECTION 1:  Signing up for Medicare17
Remember, in most cases, if you don’t sign up for Part A (if you have 
to buy it) and Part B when you’re first eligible, your enrollment may be delayed and you may have to pay a late enrollment penalty. Go to pages 22–23.
What are the Part A and Part B enrollment periods?
You can only sign up for Part B (and/or Part A if you have to buy it) during these enrollment periods.
Initial Enrollment Period 
Generally, you can first sign up for Part A and/or Part B during the 7-month period that begins 3 months before the month you turn 65 and ends 3 months after the month you turn 65. If your birthday is on the first of the month, your 7-month period starts 4 months before the month you turn 65 and ends 2 months after the month you turn 65. 
Example: If you turn 65 on June

{'input': 'Am I eligible for Medicare based on my age?',
 'chat_history': [HumanMessage(content='My name is Bob!'),
  AIMessage(content="Hi Bob, it's nice to meet you!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='I am 59 years old'),
  AIMessage(content='You are 59 years old.')],
 'output': 'I am sorry, but I cannot answer your question about Medicare eligibility based on your age of 59. The information you provided is not sufficient to determine your eligibility. To get accurate information about your Medicare eligibility, please contact Social Security at 1-800-772-1213. TTY users should call 1-800-325-0778.'}

In [62]:
agent.invoke({'input': "My spouse Wilma's age is 66"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "Is my spouse Wilma eligible for Medicare based on her age of 66?"
  }
}
```Health Insurance Marketplace® is a registered service mark of the U.S. Department of 
Health & Human Services.
“Medicare & You” isn’t a legal document. Official Medicare Program legal guidance is 
contained in the relevant statutes, regulations, and rulings.
This product was produced at U.S. taxpayer expense.
Copies of this product are available free of charge at Medicare.gov   
or by calling 1-800-MEDICARE.한국어(Korea n) 만약 귀하나 귀하가 돕는 어느 분이 메디케어에 관해서 질문을 가지고
있다면 비용 부담이 없이 필요한 도움과 정보를 귀하의 언어로 얻을 수 있는 권리가 귀하에게 
있습니다. 통역사와 말씀을 나누시려면 1-800-MEDICARE(1-800-633-4227) 로 전화하십시오.
Polsk i (Polish) Jeżeli Państwo lub ktoś komu Państwo pomagają macie pytania dotyczące Medicare, 
mają Państwo prawo do uzyskania bezpłatnej pomocy i informacji w swoim języku. Aby rozmawiać z 
tłumaczem, prosimy dzwonić p

{'input': "My spouse Wilma's age is 66",
 'chat_history': [HumanMessage(content='My name is Bob!'),
  AIMessage(content="Hi Bob, it's nice to meet you!"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='I am 59 years old'),
  AIMessage(content='You are 59 years old.'),
  HumanMessage(content='Am I eligible for Medicare based on my age?'),
  AIMessage(content='I am sorry, but I cannot answer your question about Medicare eligibility based on your age of 59. The information you provided is not sufficient to determine your eligibility. To get accurate information about your Medicare eligibility, please contact Social Security at 1-800-772-1213. TTY users should call 1-800-325-0778.')],
 'output': 'Yes, your spouse Wilma is eligible for Medicare based on her age of 66. She can enroll in Medicare three months before her 65th birthday. The initial enrollment period is seven months long, starting three months before the month she tu